In [0]:
%pip install dbl-tempo

In [0]:
#
import urllib
from pyspark.sql.functions import *
from pyspark.sql.types import *
#from tempo import *
#import dlt
import boto3
from botocore.config import Config

df = spark.sql("""SELECT * FROM machina.machina_model_v1""") # machina_cleaned # machina_raw # machina_model_v1
#display(df)

In [0]:
window = Window.partitionBy("run_uuid").orderBy("timestamp")

#df.printSchema()

#groupDF = df.groupBy("run_uuid").agg(min("timestamp").alias("min_timestamp"), \
#                                    max("timestamp").alias("max_timestamp"), \
#                                    min("unix_milliseconds").alias("min_unix_milliseconds"), \
#                                    max("unix_milliseconds").alias("max_unix_milliseconds"))
groupDF = df.groupBy("run_uuid").agg(f.min("timestamp"), \
                                    f.max("timestamp"), \
                                    f.min("unix_milliseconds"), \
                                    f.max("unix_milliseconds"))
groupDF.printSchema()
display(groupDF)
groupDF = groupDF.withColumn('total_runtime_seconds',col("max(timestamp)").cast("long") - col('min(timestamp)').cast("long")) \
    .withColumn('total_runtime_ms',col("max(unix_milliseconds)") - col('min(unix_milliseconds)')) \
    .withColumn('total_distance_traveled', f.lit(0.0)) 

groupDF = groupDF.withColumnRenamed("min(timestamp)", "min_timestamp")
groupDF = groupDF.withColumnRenamed("max(timestamp)", "max_timestamp")
groupDF = groupDF.withColumnRenamed("min(unix_milliseconds)", "min_unix_milliseconds")
groupDF = groupDF.withColumnRenamed("max(unix_milliseconds)", "max_unix_milliseconds")

display(groupDF)
groupDF.printSchema()

root
 |-- run_uuid: double (nullable = false)
 |-- min(timestamp): timestamp (nullable = true)
 |-- max(timestamp): timestamp (nullable = true)
 |-- min(unix_milliseconds): double (nullable = true)
 |-- max(unix_milliseconds): double (nullable = true)



run_uuid,min(timestamp),max(timestamp),min(unix_milliseconds),max(unix_milliseconds)
8.9100958441866568E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12
7.5822930809914696E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12
6.1769765347440763E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12
1.2405186538561671E19,2022-11-23T20:40:00.000+0000,2022-11-23T20:41:17.630+0000,1.669236E12,1.66923607763E12


run_uuid,min_timestamp,max_timestamp,min_unix_milliseconds,max_unix_milliseconds,total_runtime_seconds,total_runtime_ms,total_distance_traveled
8.9100958441866568E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12,599,599990.0,0.0
7.5822930809914696E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12,599,599990.0,0.0
6.1769765347440763E18,2022-11-23T20:40:00.000+0000,2022-11-23T20:49:59.990+0000,1.669236E12,1.66923659999E12,599,599990.0,0.0
1.2405186538561671E19,2022-11-23T20:40:00.000+0000,2022-11-23T20:41:17.630+0000,1.669236E12,1.66923607763E12,77,77630.0,0.0


root
 |-- run_uuid: double (nullable = false)
 |-- min_timestamp: timestamp (nullable = true)
 |-- max_timestamp: timestamp (nullable = true)
 |-- min_unix_milliseconds: double (nullable = true)
 |-- max_unix_milliseconds: double (nullable = true)
 |-- total_runtime_seconds: long (nullable = true)
 |-- total_runtime_ms: double (nullable = true)
 |-- total_distance_traveled: double (nullable = false)

